In [1]:
import torch 
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import os
from tqdm import  tqdm
import GPUtil as GPU
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam
import cv2
from torchvision.transforms import *
#modelling
from torch import nn 
from torch.nn import functional as F
import torch 
from torchvision import models
from pathlib import Path

In [2]:
#define the unet architecture 
def conv3x3(in_, out):
    return nn.Conv2d(in_, out, 3, padding=1)


def concat(xs):
    return torch.cat(xs, 1)


class Conv3BN(nn.Module):
    def __init__(self, in_: int, out: int, bn=False):
        super().__init__()
        self.conv = conv3x3(in_, out)
        self.bn = nn.BatchNorm2d(out) if bn else None
        self.activation = nn.SELU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        if self.bn is not None:
            x = self.bn(x)
        x = self.activation(x)
        return x

    
class DecoderBlock(nn.Module):
    def __init__(self, in_channels, middle_channels, out_channels):
        super().__init__()

        self.block = nn.Sequential(
            ConvRelu(in_channels, middle_channels),
            nn.ConvTranspose2d(middle_channels, out_channels, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.block(x)
    

class ConvRelu(nn.Module):
    def __init__(self, in_: int, out: int):
        super().__init__()
        self.conv = conv3x3(in_, out)
        self.activation = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.activation(x)
        return x


In [3]:

class UNet11(nn.Module):
    def __init__(self, num_classes=1, num_filters=32):
        super().__init__()
        self.pool = nn.MaxPool2d(2, 2)
        self.encoder = models.vgg11(pretrained=True).features
        self.relu = self.encoder[1]
        self.conv1 = self.encoder[0]
        self.conv2 = self.encoder[3]
        self.conv3s = self.encoder[6]
        self.conv3 = self.encoder[8]
        self.conv4s = self.encoder[11]
        self.conv4 = self.encoder[13]
        self.conv5s = self.encoder[16]
        self.conv5 = self.encoder[18]

        self.center = DecoderBlock(num_filters * 8 * 2, num_filters * 8 * 2, num_filters * 8)
        self.dec5 = DecoderBlock(num_filters * (16 + 8), num_filters * 8 * 2, num_filters * 8)
        self.dec4 = DecoderBlock(num_filters * (16 + 8), num_filters * 8 * 2, num_filters * 4)
        self.dec3 = DecoderBlock(num_filters * (8 + 4), num_filters * 4 * 2, num_filters * 2)
        self.dec2 = DecoderBlock(num_filters * (4 + 2), num_filters * 2 * 2, num_filters)
        self.dec1 = ConvRelu(num_filters * (2 + 1), num_filters)

        self.final = nn.Conv2d(num_filters, num_classes, kernel_size=1)

    def forward(self, x):
        conv1 = self.relu(self.conv1(x))
        conv2 = self.relu(self.conv2(self.pool(conv1)))
        conv3s = self.relu(self.conv3s(self.pool(conv2)))
        conv3 = self.relu(self.conv3(conv3s))
        conv4s = self.relu(self.conv4s(self.pool(conv3)))
        conv4 = self.relu(self.conv4(conv4s))
        conv5s = self.relu(self.conv5s(self.pool(conv4)))
        conv5 = self.relu(self.conv5(conv5s))

        center = self.center(self.pool(conv5))

        dec5 = self.dec5(torch.cat([center, conv5], 1))
        dec4 = self.dec4(torch.cat([dec5, conv4], 1))
        dec3 = self.dec3(torch.cat([dec4, conv3], 1))
        dec2 = self.dec2(torch.cat([dec3, conv2], 1))
        dec1 = self.dec1(torch.cat([dec2, conv1], 1))
        return F.sigmoid(self.final(dec1))

    
#writing the dice based loss weights for the neural network 

class Loss:
    #init script for the loss
    def __init__(self, dice_weight=1):
        self.nll_loss = nn.BCELoss()
        self.dice_weight = dice_weight

    #call called if parameters are given to the network 
    def __call__(self, outputs, targets):
        loss = self.nll_loss(outputs, targets)
        if self.dice_weight:
            eps = 1e-15
            dice_target = (targets == 1).float()
            dice_output = outputs
            intersection = (dice_output * dice_target).sum()
            union = dice_output.sum() + dice_target.sum() + eps

            loss -= torch.log(2 * intersection / union)

        return loss


In [4]:
########Normal Image Transforms#########################################

img_transform = Compose([
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
def load_image(path):
    img=cv2.imread(path)
    img = cv2.copyMakeBorder(img, 0, 0, 1, 1, cv2.BORDER_REFLECT_101)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #img=cv2.resize(img,(img.shape[1]//2,img.shape[0]//2))
    return img.astype(np.uint8)

#dataset to load the prediction images
class PredictionDatasetPure:
    def __init__(self, paths):
        self.paths = paths

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx % len(self.paths)]
        #print(path)
        image = load_image(str(path))
        #path.stem returns the path 
        return img_transform(image),path.stem


In [5]:
#create the prediction directory here
local_data_path=Path('.').absolute()
prediction_path=(local_data_path/'predictions')
prediction_path.mkdir(exist_ok=True, parents=True)
#load the test images
test_images=sorted(list((local_data_path/'test_hq').glob('*.jpg')))
len_test=len(test_images)

In [12]:
#function to load images from_path, make preductions to to_path

def predict(model, from_path,to_path,batch_size=2):
    loader = DataLoader(
            dataset=PredictionDatasetPure(from_path),
            shuffle=False,
            batch_size=batch_size,
            num_workers=1,
            pin_memory=True
        )
    for i,(inputs, stems) in enumerate(tqdm(loader,desc='Prediction Progress')):
        print(i)
        #inputs=inputs.to(device)
        outputs=model(inputs)
        with torch.no_grad():
            mask=(outputs.detach().cpu().numpy()*255).astype(np.uint8)
            #print(mask)
            #print(mask.shape)
            #print("path is",str(to_path/(stems[0]+'.png')))
            cv2.imwrite(str(to_path/(stems[0]+'.png')),mask[0,0,:,:])

In [13]:
class WrappedModel(nn.Module):
	def __init__(self, module):
		super(WrappedModel, self).__init__()
		self.module = module # that I actually define.
	def forward(self, x):
		return self.module(x)



In [14]:
device = torch.device('cpu')
fold_no=0
#create test outputs corresponding to each fold in the prediction directory
fold_path=prediction_path/str(fold_no)
fold_path.mkdir(exist_ok=True, parents=True)

test_path=fold_path/'test'
test_path.mkdir(exist_ok=True,parents=True)

#predict from test_images, copy to test_path
#load the model here 

model=UNet11()
model=WrappedModel(model)
device = torch.device('cpu')
#device = torch.device('cpu')
#model=UNet11()
#model = nn.DataParallel(model, device_ids=[])
model_checkpoint='model_'+str(fold_no)+'.pt'
model_checkpoint_path=local_data_path/'bestmodelweights'/model_checkpoint

#load_the_model_weights

state=torch.load(model_checkpoint_path,map_location=device)
model.load_state_dict(state['model'])
model.eval()

predict(model,test_images,test_path,1)



Prediction Progress:   0%|          | 0/100064 [00:00<?, ?it/s]

0




Prediction Progress:   0%|          | 1/100064 [00:13<372:19:29, 13.40s/it]

1




Prediction Progress:   0%|          | 2/100064 [00:26<370:38:45, 13.33s/it]

...



Prediction Progress:   0%|          | 54/100064 [09:31<261:49:16,  9.42s/it]

54




Prediction Progress:   0%|          | 55/100064 [09:40<259:14:32,  9.33s/it]

55


KeyboardInterrupt: 

In [ ]:
test_images

In [ ]:
print(state['best_valid_loss'])

In [ ]:
device

In [ ]:
device